In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import linregress

**Discovery**
-------------
Discovery is the exploratory phase of the data analysis. Learning how to cleanse the data to produce meaningful results during analysis

In [2]:
# combine all the data/usda_* into one dataframe
usda = pd.concat([pd.read_csv(f'data/usda_data_{year}.csv')  for year in range(1950, 2024)])

# combine all the data/weather_* into one dataframe
weather = pd.concat([pd.read_csv(f'data/weather_data_{year}.csv') for year in range(1950, 2024)])


In [ ]:
usda.info()

In [ ]:
weather.info()

In [ ]:
# what is the average value in udsa data for each year
usda.groupby('year')['Value'].mean()


In [ ]:
# what is the average value in udsa data for each year
usda.groupby('year')['Value'].sum()

Selecting the columns needed for analysis (year, county_name, county_ansi and Value).

In [7]:
usda['year'] = pd.to_datetime(usda['year'],
               format='%Y')
usda_subset = usda[['year','county_name','county_ansi','Value']]

In [ ]:
usda_subset['county_name'].nunique() #85 Michigan has 83

The result is that the data set has 85 and Michigan only has 83. The additional two are 'OTHER COUNTIES' and 'OTHER (COMBINED) COUNTIES'.
what do the entries look like for 'OTHER (COMBINED) COUNTIES'?

In [ ]:
usda_subset_other_combined = usda_subset[usda_subset['county_name'] == 'OTHER (COMBINED) COUNTIES']
usda_subset_other_combined

There are 144 records that contain 'OTHER (COMBINED) COUNTIES' and three entries for 1960 and several entries for 2019. It appears that perhaps the "Other Counties" was a dumping ground for smaller quanities collected over several counties.

The 3 entries for 1960 have the following asd_desc: "UPPER PENINSULA", "NORTHWEST", and "NORTHEAST". The location_desc has these entries: "MICHIGAN, UPPER PENINSULA, OTHER (COMBINED) COUNTIES", "MICHIGAN, NORTHWEST, OTHER (COMBINED) COUNTIES", and "MICHIGAN, NORTHEAST, OTHER (COMBINED) COUNTIES". How many bushels/acre were reported total?

In [ ]:
pd.set_option("display.max_columns", None)
usda_subset_1960 = usda[(usda['year'] == '1960-01-01') & usda['county_name'].str.upper().str.contains('OTHER')]
usda_subset_1960

In [ ]:
usda_subset_1960.groupby('county_name')['Value'].sum().reset_index()

The total Value for 1960 "OTHER (COMBINED) COUNTIES" was 130.6
A similiar dumping ground value when compared to the single "OTHER COUNTIES" in 2020.

2019 seemed to have a lot of entries for "OTHER (COMBINED) COUNTIES". What is going on?

In [ ]:
usda_2019 = usda[(usda['year'] == '2019-01-01') & (usda['county_name'] == 'OTHER (COMBINED) COUNTIES')]
usda_2019

The 7 entries for 2019 have the following asd_desc: "UPPER PENINSULA", "NORTHWEST", "NORTHEAST","WEST CENTRAL", "CENTRAL", "SOUTHWEST", and "SOUTH CENTRAL". The location_desc has these entries: "MICHIGAN, UPPER PENINSULA, OTHER (COMBINED) COUNTIES", "MICHIGAN, NORTHWEST, OTHER (COMBINED) COUNTIES", "MICHIGAN, NORTHEAST, OTHER (COMBINED) COUNTIES","MICHIGAN, WEST CENTRAL, OTHER (COMBINED) COUNTIES", "MICHIGAN, CENTRAL, OTHER (COMBINED) COUNTIES", "MICHIGAN, SOUTHWEST, OTHER (COMBINED) COUNTIES", and "MICHIGAN, SOUTH CENTRAL, OTHER (COMBINED) COUNTIES".

How many years and how many entries are we looking at that are like this? It looks like 2019 was the worst with 7 regions identified. The amount of bushels/acre is  886.3. This is considerably more than 1960.

In [ ]:
grouped_df = usda.groupby(['year', 'county_name']).agg(
    count=('county_name', 'count'),
    sum=('Value', 'sum')
).reset_index()

filtered_df = grouped_df[grouped_df['count'] > 1]
# Sort by 'sum' in descending order
filtered_df = filtered_df.sort_values(by='sum', ascending=False)

print(filtered_df)

When comparing to the rest of the entries in 2019. How bad does it look?

In [14]:
usda_subset_2019=usda_subset[usda_subset['year'] == '2019-01-01']
grouped_df_2019 = usda_subset_2019.groupby(['year', 'county_name']).agg(
    count=('county_name', 'count'),
    sum=('Value', 'sum')
).reset_index()

In [ ]:
# Adjust figure size for better readability
plt.figure(figsize=(12, 6))

# Create the scatter plot
g = sns.scatterplot(data=grouped_df_2019, x="county_name", y="sum", hue="year")

# Rotate the x-axis labels for readability
plt.xticks(rotation=90)

# Add labels and title
plt.xlabel('Counties')
plt.ylabel('BU/Acre')
plt.title('Bushels/Acre by County for 2019')

# Adjust the legend positioning
#g.legend(loc='center left', bbox_to_anchor=(1.25, 0.5), ncol=1)

# Display the plot
plt.show()

The rest of the counties for 2019 produced no more than 200 bushels/acre. As expected, "OTHER (COMBINED) COUNTIES" is the highest in the chart. What about those single entries for 'OTHER COUNTIES' after 2020?

In [ ]:
usda_subset_other = usda_subset[usda_subset['county_name'] == 'OTHER COUNTIES']
usda_subset_other

There are 4 records (years) that contain 'OTHER COUNTIES'; no duplicated years. We will go back to the original dataset and look at the entire record for those years.

In [ ]:
pd.set_option("display.max_columns", None)
usda_other = usda[usda['county_name'] == 'OTHER COUNTIES']
usda_other

The asd_desc and location_desc is null for all 4 records is "MICHIGAN, OTHER COUNTIES".

We will take a look and see how many total counties are reporting beyond 2020.

In [ ]:
# Filter the DataFrame for the year >= 2020
usda_subset = usda[usda['year'] >= '2020']

# Count the number of unique county names by year
unique_counties_by_year = usda_subset.groupby('year')['county_name'].nunique()
unique_counties_by_year

There are 61 counties in the data set for 2020. And those unique counties are:

In [ ]:
# Filter the DataFrame for the year >= 2020
usda_subset_2020 = usda_subset[usda_subset['year'] == '2020']
usda_subset_2020['county_name'].unique()

array(['DELTA', 'DICKINSON', 'MENOMINEE', 'ANTRIM', 'BENZIE',
       'CHARLEVOIX', 'EMMET', 'GRAND TRAVERSE', 'LEELANAU', 'MANISTEE',
       'WEXFORD', 'ALCONA', 'ALPENA', 'IOSCO', 'OGEMAW', 'OTSEGO',
       'PRESQUE ISLE', 'MASON', 'MUSKEGON', 'NEWAYGO', 'OCEANA',
       'GLADWIN', 'ISABELLA', 'MECOSTA', 'MIDLAND', 'MONTCALM', 'ARENAC',
       'BAY', 'HURON', 'SAGINAW', 'SANILAC', 'TUSCOLA', 'ALLEGAN',
       'BERRIEN', 'CASS', 'KALAMAZOO', 'KENT', 'OTTAWA', 'VAN BUREN',
       'BARRY', 'BRANCH', 'CALHOUN', 'CLINTON', 'EATON', 'HILLSDALE',
       'INGHAM', 'IONIA', 'JACKSON', 'ST JOSEPH', 'SHIAWASSEE', 'GENESEE',
       'LAPEER', 'LENAWEE', 'LIVINGSTON', 'MACOMB', 'MONROE', 'OAKLAND',
       'ST CLAIR', 'WASHTENAW', 'WAYNE', 'OTHER COUNTIES'], dtype=object)

In [ ]:
# what is the average value in udsa data for each of those years
mean_value = usda_subset.groupby('year')['Value'].mean()

# create other Dataframe from orignial data
usda_other = usda[usda['county_name'] == 'OTHER COUNTIES']
usda_other

# Merge the two DataFrames for comparison
comparison = pd.merge(mean_value, usda_other, on='year', how='outer')

# Rename columns for clarity
comparison.rename(columns={'Value_x': 'Yearly Mean', 'Value_y':'Value'}, inplace=True)

# Calculate the difference
comparison['Difference'] = comparison['Yearly Mean'] - comparison['Value']
comparison[['year','Yearly Mean', 'Value', 'Difference']]

The difference is no greater than 14.7 bushels per acre. This is not significant when compared to the mean for each year.

Now let's do the same for the "OTHER (COMBINED)" years. They stopped using that in 2019.

In [ ]:
# Filter the DataFrame for the year <= 2019
usda_subset_other_combined = usda[usda['year'] <= '2019']

# what is the average value in udsa data for each of those years
mean_value = usda_subset_other_combined.groupby('year')['Value'].mean()

# Filter for counties where 'county_name' contains 'OTHER (COMBINED)' (case insensitive)
other_combined = usda_subset_other_combined[usda_subset_other_combined['county_name'].str.contains('OTHER \(COMBINED\)', case=False, regex=True)]

# Sum the 'Value' for OTHER (COMBINED) COUNTIES
other_combined_sum = other_combined.groupby('year')['Value'].sum().reset_index('year')

# Merge the two DataFrames for comparison and drop the records where there weren't "Other (Combined)" Counties
comparison = pd.merge(mean_value, other_combined_sum, on='year', how='outer').dropna()

# Rename columns for clarity
comparison.rename(columns={'Value_x': 'Yearly Mean', 'Value_y':'Other Total Value'}, inplace=True)

# Calculate the difference
comparison['Difference'] = comparison['Yearly Mean'] - comparison['Other Total Value']
comparison[['year','Yearly Mean', 'Other Total Value', 'Difference']]

In [ ]:
# Set the figure size
plt.figure(figsize=(10, 6))

# Plot the yearly mean and total value for "Other Combined" counties
sns.lineplot(x='year', y='Yearly Mean', data=comparison, label='Yearly Mean')
sns.lineplot(x='year', y='Other Total Value', data=comparison, label='Other Total Value')

# Customize the plot
plt.title('Yearly Mean vs. Other Combined Counties Total Value')
plt.xlabel('Year')
plt.ylabel('Value')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()

# Show plot
plt.show()

The average value across counties does not change much over time, remaining stable even as the data for "Other Combined" counties fluctuates. The Other total value has several sharp spikes, with rapid increases and decreases.

Now that we understand how to cleanse the usda data, we will create a function.

In [23]:
def clean_usda(usda_df, start_year, end_year, other='Yes'):
    # We will focus the fields from the dataframe we are interested in
    filtered_df = usda_df[['year','county_name','state_ansi','county_ansi','Value']]

    # We will change the datatype of year to datetime because it fits the data and we can have access to date functions
    filtered_df.loc[:,'year'] = pd.to_datetime(filtered_df['year'],
               format='%Y')
    
    # We need to ensure the start_year is a year too.
    start_year = pd.to_datetime(str(start_year), format='%Y')

    if end_year is not None:
        end_year = pd.to_datetime(str(end_year), format='%Y')
    else: end_year = start_year

    # Filter starting from the requested year
    filtered_df = filtered_df[(filtered_df['year'] >= start_year) & (filtered_df['year'] <= end_year)]
   
    # Remove the "Other" type county entries. We convert to all caps to handle any inconsistencies
    if other == 'Yes':
        filtered_df = filtered_df[~filtered_df['county_name'].str.upper().str.contains('OTHER')]
    
    return filtered_df

Try out the new function.

In [ ]:
usda_subset = clean_usda(usda,1950,2024,)
usda_subset

Make a cool looking graph.

In [ ]:
import matplotlib.patches as mpatches

plt.figure(figsize=(15, 8))

# Create the plot
ax = sns.histplot(data=usda_subset, x="Value", hue="county_name", multiple="stack", palette="tab20")

plt.xlabel('BU/Acre')
plt.title('Bushels/Acre by County for Past 7 Decades')

# Manually create the legend with the right colors from the palette
# Get the palette used for the plot
palette = sns.color_palette("tab20", n_colors=usda_subset['county_name'].nunique())

# Get unique counties and map them to the corresponding palette color
handles = [mpatches.Patch(color=palette[i], label=county) 
           for i, county in enumerate(usda_subset['county_name'].unique())]

# Add the legend outside the plot
ax.legend(handles=handles, bbox_to_anchor=(1.05, 1), loc='upper left', 
          borderaxespad=0, ncol=2, fontsize='small')

plt.tight_layout()
plt.show()


In [26]:
# GDD calculation function
def calculate_gdd(df, base_temp=50, upper_temp=86):
    """
    Calculate Growing Degree Days (GDD) for corn.
    """
    df['TMAX'] = df['TMAX'].clip(lower=base_temp, upper=upper_temp)
    df['TMIN'] = df['TMIN'].clip(lower=base_temp)
    df['TAVG'] = (df['TMAX'] + df['TMIN']) / 2
    df['GDD'] = df['TAVG'] - base_temp
    return df

In [27]:
weather = calculate_gdd(weather)

In [ ]:
# Compare average yield for usda_data_1955 and usda_data_2015
usda_1955 = clean_usda(usda,1955,None,)
usda_2015 = clean_usda(usda,2015,None,)

usda_1955['Value'].mean(), usda_2015['Value'].mean()

# whats the std deviation of yield for those years
usda_1955['Value'].std(), usda_2015['Value'].std()


In [ ]:
# Compare total gdd for weather_data_1955 and weather_data_2015 for county_ansi=161
weather_2014 = pd.read_csv('data/weather_data_2014.csv')
weather_2023 = pd.read_csv('data/weather_data_2023.csv')

weather_2014 = calculate_gdd(weather_2014)
weather_2023 = calculate_gdd(weather_2023)

weather_2014[weather_2014['county_ansi'] == 161]['GDD'].sum(), weather_2023[weather_2023['county_ansi'] == 161]['GDD'].sum()

# Do this for all years between 1950 and 1959
gdd = []
for year in range(1950, 1960):
    weather = pd.read_csv(f'data/weather_data_{year}.csv')
    weather = calculate_gdd(weather)
    gdd.append(weather[weather['county_ansi'] == 161]['GDD'].sum())

gdd
